In [2]:
import pandas as pd
import sqlite3
from sqlite3 import Error

def create_connection():
    conn = None;
    try:
        conn = sqlite3.connect('proj.db') 
        print(f'Successful Connection with SQLite version {sqlite3.version}')
    except Error as e:
        print(f'The error {e} occurred')

    return conn

def create_table_from_csv(conn, csv_file_path, table_name, datetime_columns=None):
    # read csv file with correct encoding
    df = pd.read_csv(csv_file_path) # or 'euc-kr'

    if datetime_columns:
        for col in datetime_columns:
            # convert datetime columns to datetime
            df[col] = pd.to_datetime(df[col])

            # remove any rows that have null dates
            df = df.dropna(subset=[col])

    # replace any whitespace in column names with '_'
    df.columns = df.columns.str.replace(' ', '_')

    # create table in sqlite db
    df.to_sql(table_name, conn, if_exists='replace', index=False)

    print(f'Table {table_name} has been created.')

# create a db connection
connection = create_connection()

# create tables from csv files
create_table_from_csv(connection, 'preprocessed_hotel 수정.csv', '호텔')
create_table_from_csv(connection, 'preprocessed_cars 수정.csv', '렌터카')
create_table_from_csv(connection, 'preprocessesd_flight 수정.csv', '항공권', ['departure_datetime', 'arrival_datetime'])



Successful Connection with SQLite version 2.6.0
Table 호텔 has been created.
Table 렌터카 has been created.
Table 항공권 has been created.


In [2]:
# Reconnect to the SQLite database
conn = sqlite3.connect('proj.db')
cur = conn.cursor()

# Create a new table with the same structure as the '호텔' table, but with 'start_date' and 'end_date' as TIMESTAMP
cur.execute("""
CREATE TABLE 호텔_new (
    hotel_name TEXT,
    region TEXT,
    ratings REAL,
    price INTEGER,
    start_date TIMESTAMP,
    end_date TIMESTAMP,
    capacity INTEGER,
    new_region TEXT,
    금전_상황 TEXT
);
""")

# Copy all rows from the '호텔' table to the '호텔_new' table
cur.execute("""
INSERT INTO 호텔_new
SELECT * FROM 호텔;
""")

# Drop the old '호텔' table
cur.execute("DROP TABLE 호텔;")

# Rename the '호텔_new' table to '호텔'
cur.execute("ALTER TABLE 호텔_new RENAME TO 호텔;")

# Commit the changes and close the connection
conn.commit()
cur.close()
conn.close()


In [3]:
conn = sqlite3.connect('proj.db')
cur = conn.cursor()

# Create a new table with the same structure as the '렌터카' table, but with 'start_date' and 'end_date' as TIMESTAMP
cur.execute("""
CREATE TABLE 렌터카_new (
    car_name TEXT,
    brand_name TEXT,
    seats INTEGER,
    size TEXT,
    fuel_type TEXT,
    transmission_type TEXT,
    rental_company_name TEXT,
    age_req INTEGER,
    driving_experience INTEGER,
    year TEXT,
    ratings REAL,
    num_ratings INTEGER,
    price INTEGER,
    start_date TIMESTAMP,
    end_date TIMESTAMP,
    금전_상황 TEXT
);
""")

# Copy all rows from the '렌터카' table to the '렌터카_new' table
cur.execute("""
INSERT INTO 렌터카_new
SELECT * FROM 렌터카;
""")

# Drop the old '렌터카' table
cur.execute("DROP TABLE 렌터카;")

# Rename the '렌터카_new' table to '렌터카'
cur.execute("ALTER TABLE 렌터카_new RENAME TO 렌터카;")

# Commit the changes and close the connection
conn.commit()
cur.close()
conn.close()